In [24]:
from dotenv import load_dotenv

load_dotenv()

import io
from importlib import reload

from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.core.credentials import AzureKeyCredential
from fastapi import UploadFile
from pillow_heif import register_heif_opener

from api.internal.receipt import extract_receipt_details
from api.schemas import receipt
from api.settings import settings

In [2]:
client = DocumentIntelligenceClient(
    endpoint=str(settings.AZURE_AI_ENDPOINT),
    credential=AzureKeyCredential(settings.AZURE_AI_ACCESS_KEY),
)

In [3]:
fh = open(settings.BASE_DIR.parent / "data" / "receipts" / "IMG_6102.HEIC", "rb")
file = UploadFile(file=fh, filename="IMG_6102.HEIC")

with io.BytesIO(file.file.read()) as buf:
    buf.seek(0)
    poller = client.begin_analyze_document(
        "prebuilt-receipt",
        analyze_request=buf,
        content_type="application/octet-stream",
    )
    res = poller.result()

In [44]:
res.documents[0].fields["TaxDetails"].value_array[0]

{'type': 'object', 'valueObject': {'Amount': {'type': 'currency', 'valueCurrency': {'amount': 0.94, 'currencyCode': 'GBP'}, 'content': '0.94', 'boundingRegions': [{'pageNumber': 1, 'polygon': [1063, 1863, 1214, 1862, 1214, 1928, 1063, 1928]}], 'confidence': 0.981, 'spans': [{'offset': 209, 'length': 4}]}}, 'content': '0.94 VAT 20%\n5.65', 'boundingRegions': [{'pageNumber': 1, 'polygon': [1063, 1836, 2093, 1840, 2093, 1932, 1063, 1928]}], 'confidence': 0.98, 'spans': [{'offset': 209, 'length': 17}]}

In [32]:
reload(receipt)

print(receipt.Receipt.model_validate(res.documents[0]).model_dump_json(indent=2))

{
  "receipt_type": "receipt.retailMeal",
  "total": 8.15,
  "total_tax": 0.94,
  "tax_details": 0.94,
  "merchant_address": "47 49 Tooley Street SE1 2QN",
  "merchant_name": "Pret A Manger (Europe) Ltd",
  "transaction_date": "2024-10-04",
  "transaction_time": "14:58:00",
  "items": [
    {
      "description": "TA Toastie Classic Cheese",
      "quantity": 1,
      "total_price": 5.65
    },
    {
      "description": "TA Very Berry Croissant",
      "quantity": 1,
      "total_price": 2.5
    }
  ]
}


In [4]:
register_heif_opener()

with open(settings.BASE_DIR.parent / "data" / "receipts" / "IMG_6102.HEIC", "rb") as fh:
    res = await extract_receipt_details(fh)

print(res.model_dump_json(indent=2))

'{\n  "receipt_type": "receipt.retailMeal",\n  "items": [\n    {\n      "description": "TA Toastie Classic Cheese",\n      "quantity": 1,\n      "total_price": 5.65\n    },\n    {\n      "description": "TA Very Berry Croissant",\n      "quantity": 1,\n      "total_price": 2.5\n    }\n  ],\n  "total": 8.15,\n  "total_tax": 0.94,\n  "tax_details": 0.94,\n  "merchant_address": "47 49 Tooley Street SE1 2QN",\n  "merchant_name": "Pret A Manger (Europe) Ltd",\n  "transaction_date": "2024-10-04",\n  "transaction_time": "14:58:00"\n}'

In [5]:
print(res.model_dump_json(indent=2))

{
  "receipt_type": "receipt.retailMeal",
  "items": [
    {
      "description": "TA Toastie Classic Cheese",
      "quantity": 1,
      "total_price": 5.65
    },
    {
      "description": "TA Very Berry Croissant",
      "quantity": 1,
      "total_price": 2.5
    }
  ],
  "total": 8.15,
  "total_tax": 0.94,
  "tax_details": 0.94,
  "merchant_address": "47 49 Tooley Street SE1 2QN",
  "merchant_name": "Pret A Manger (Europe) Ltd",
  "transaction_date": "2024-10-04",
  "transaction_time": "14:58:00"
}
